## <span style="color:#c70d39">**Proyecto Final**</span>
### <span style="color:#c70d39">**Desarrollo de un Modelo de ML para Predecir la Severidad de los Accidentes en USA**</span>

#### <span style="color:#ff5733">**Librerías a utilizar**</span>

In [2]:
import tensorflow as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detectar TPU
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy()  # Estrategia predeterminada para CPU y GPU
print("Número de dispositivos: {}".format(strategy.num_replicas_in_sync))


E0000 00:00:1723134754.206206      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0808 16:32:34.214264953      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0808 16:32:34.214280387      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0808 16:32:34.214283690      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0808 16:32:34.214286090      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0808 16:32:34.214288450      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1723134767.804875      13 service.cc:145] XLA service 0x5774173dbb80 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723134767.804936      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1723134767.804940      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1723134767.804944      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1723134767.804947      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1723134767.804949      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1723134767.804952      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1723134767.804954      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1723134767.804957      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [3]:
!pip install scikeras


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install dill joblib
!pip install keras-tuner




[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sqlite3
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasClassifier
from scipy.stats import randint
import dill as pickle
import keras_tuner as kt


### <span style="color:#c70d39">**Cargando Data Preprocesada (Train y Test)**</span>

In [6]:
conn = sqlite3.connect('/kaggle/input/accidentes-usa-db/accidentes_usa.db')
cursor_obj = conn.cursor()

# Cargar los conjuntos de datos
X_train = pd.read_sql('SELECT * FROM X_train', conn)
X_test = pd.read_sql('SELECT * FROM X_test', conn)
y_train = pd.read_sql('SELECT * FROM y_train', conn)
y_test = pd.read_sql('SELECT * FROM y_test', conn)

# Cerrar la conexión
conn.close()

In [7]:
X_train.head()

,Start_Lat,Start_Lng,Traffic_Signal,Civil_Twilight,Temperature(F),Humidity(%)
0,0.582115,0.846743,0.0,1.0,0.515152,0.938776
1,0.717055,0.863890,0.0,1.0,0.323232,0.959184
2,0.488287,0.667733,0.0,1.0,0.212121,0.724490
3,0.543816,0.057136,0.0,1.0,0.555556,0.642857
4,0.664801,0.917162,0.0,1.0,0.363636,0.857143


In [8]:
# Convirtiendo la clase en series para correr en los modelos a evaluar
y_train=y_train['Severity']
y_test=y_test['Severity']

### Modelo Multiclase con Redes Neuronales

In [9]:
# Convertir y_train y y_test a formato one-hot
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
# Calcular el número de clases
num_clases = y_train_cat.shape[1]

# Configurar la estrategia para usar la GPU
strategy = tf.distribute.MirroredStrategy()  # Usar todas las GPUs disponibles
print("Número de dispositivos: {}".format(strategy.num_replicas_in_sync))

from tensorflow.keras.optimizers import Adam

with strategy.scope():
    model = Sequential([
        Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(num_clases, activation='softmax')
    ])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train_cat, epochs=100, batch_size=32, validation_split=0.2)
    
# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test_cat)
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")    

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Número de dispositivos: 1
Epoch 1/100


/usr/local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1632/1632 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4197 - loss: 1.2701 - val_accuracy: 0.4922 - val_loss: 1.1532
Epoch 2/100
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4804 - loss: 1.1775 - val_accuracy: 0.5064 - val_loss: 1.1226
Epoch 3/100
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4891 - loss: 1.1545 - val_accuracy: 0.5224 - val_loss: 1.0995
Epoch 4/100
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5078 - loss: 1.1283 - val_accuracy: 0.5303 - val_loss: 1.0853
Epoch 5/100
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5202 - loss: 1.1095 - val_accuracy: 0.5480 - val_loss: 1.0666
Epoch 6/100
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5255 - loss: 1.0960 - val_accuracy: 0.5458 - val_loss: 1.0587
Epoch 7/100
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5270 - loss: 1.0908 - val_accuracy: 0.5547 - val_loss: 1.0475
Epoch 8/100
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5324 - loss: 1.0823 - val_

### Optimizacion de Modelo.

In [10]:

strategy = tf.distribute.MirroredStrategy()  # Usar todas las GPUs disponibles
print("Número de dispositivos: {}".format(strategy.num_replicas_in_sync))
def build_model(hp):
    with strategy.scope():
        model = Sequential([
            Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
            Dropout(0.5),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(64, activation='relu'),
            Dense(num_clases, activation='softmax')
        ])
        learning_rate = hp.Choice('learning_rate', values=[0.001, 0.01, 0.1])
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='intro_to_kt'
)

tuner.search_space_summary()

tuner.search(X_train, y_train_cat, epochs=50, validation_split=0.2)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Mejores hiperparámetros encontrados: {best_hps.values}")


Trial 3 Complete [00h 03m 05s]
val_accuracy: 0.48046875

Best val_accuracy So Far: 0.5716145634651184
Total elapsed time: 00h 09m 17s
Mejores hiperparámetros encontrados: {'learning_rate': 0.001}


In [12]:
RN_Best_Model = build_model(best_hps)

with strategy.scope():
    RN_Best_Model.fit(X_train, y_train_cat, epochs=50, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = RN_Best_Model.evaluate(X_test, y_test_cat)
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")



Epoch 1/50


/usr/local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1632/1632 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4176 - loss: 1.2701 - val_accuracy: 0.4871 - val_loss: 1.1646
Epoch 2/50
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4796 - loss: 1.1750 - val_accuracy: 0.5062 - val_loss: 1.1281
Epoch 3/50
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4966 - loss: 1.1501 - val_accuracy: 0.5221 - val_loss: 1.1048
Epoch 4/50
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5064 - loss: 1.1297 - val_accuracy: 0.5352 - val_loss: 1.0867
Epoch 5/50
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5107 - loss: 1.1172 - val_accuracy: 0.5447 - val_loss: 1.0735
Epoch 6/50
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5177 - loss: 1.1076 - val_accuracy: 0.5411 - val_loss: 1.0564
Epoch 7/50
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5212 - loss: 1.0956 - val_accuracy: 0.5483 - val_loss: 1.0557
Epoch 8/50
1632/1632 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5304 - loss: 1.0868 - val_accurac

In [ ]:
# Guardando el modelo en un archivo
with open('modelo.pkl', 'wb') as file:
    pickle.dump(RN_Best_Model, file)
